In [1]:
import pandas as pd
import folium
from pyproj import Transformer

In [4]:
city = 'graz'
stations = pd.read_csv('../data/' + city + '/stations/netatmo_stations.csv', delimiter=',')
stations.head()

,device_id,module_id,lon,lat
0,70:ee:50:37:23:aa,02:00:00:36:fd:2e,15.240849,47.046135
1,70:ee:50:05:06:58,02:00:00:04:81:7a,15.271094,47.057315
2,70:ee:50:7f:a6:ec,02:00:00:7f:a3:62,15.280108,47.060491
3,70:ee:50:a5:80:90,02:00:00:a4:2f:5e,15.285259,47.056542
4,70:ee:50:28:bc:4c,02:00:00:27:1a:0c,15.282566,47.062895


In [5]:
df = pd.read_csv('../data/' + city + '/stations/temperatures.csv', delimiter=',')
df.head()

,date,time,temp,station
0,2023-06-01,1,18.0,70:ee:50:01:19:10
1,2023-06-01,2,17.1,70:ee:50:01:19:10
2,2023-06-01,3,16.5,70:ee:50:01:19:10
3,2023-06-01,4,16.0,70:ee:50:01:19:10
4,2023-06-01,5,18.2,70:ee:50:01:19:10


In [3]:
mean_temp = df.groupby('station')['temp'].mean().reset_index()
mean_temp.columns = ['station', 'mean_temp']
mean_temp.head()

,station,mean_temp
0,70:ee:50:01:19:10,21.626100
1,70:ee:50:01:27:84,22.581890
2,70:ee:50:01:29:30,23.133154
3,70:ee:50:01:5e:8a,21.000733
4,70:ee:50:01:f0:1e,20.893548


In [7]:
plot_df = mean_temp.merge(stations, left_on='station', right_on='device_id')
plot_df.head()

,station,mean_temp,device_id,module_id,lon,lat
0,70:ee:50:01:19:10,21.626100,70:ee:50:01:19:10,02:00:00:01:38:54,15.453963,46.982571
1,70:ee:50:01:27:84,22.581890,70:ee:50:01:27:84,02:00:00:01:35:22,15.336037,47.000615
2,70:ee:50:01:29:30,23.133154,70:ee:50:01:29:30,02:00:00:65:0f:74,15.566700,47.083300
3,70:ee:50:01:5e:8a,21.000733,70:ee:50:01:5e:8a,02:00:00:01:2f:d6,15.431526,47.049806
4,70:ee:50:01:f0:1e,20.893548,70:ee:50:01:f0:1e,02:00:00:2b:52:6a,15.294358,47.005366


In [16]:
from branca.colormap import LinearColormap

colors = ["#3b4cc0", "#f7f7f7", "#b40426"] # blue → white → red
colormap = LinearColormap(
    colors=colors,
    vmin=plot_df.mean_temp.min(),
    vmax=plot_df.mean_temp.max(),
    caption="Mean temperature (°C)"
)

In [17]:
# Create a map
m = folium.Map(
    location=[plot_df.lat.mean(), plot_df.lon.mean()],
    zoom_start=13,
    control_scale=True)

# Add markers
for _, row in plot_df.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=8,
        popup=(f"Station: {row.station}<br>"
               f"Mean temp: {row.mean_temp:.2f} °C"),
        color=colormap(row.mean_temp),
        fill=True,
        fill_opacity=0.9
    ).add_to(m)

# Add a legend
colormap.caption = "Mean temperature (°C)"
colormap.add_to(m)

# Display
m

In [19]:
# Print stations with lowest temperatures
lowest = (mean_temp.sort_values('mean_temp').head(5))
print(lowest)

               station  mean_temp
96   70:ee:50:73:e9:4c  18.659685
71   70:ee:50:37:20:4a  18.782894
106  70:ee:50:7a:fa:08  18.915731
119  70:ee:50:90:f2:36  19.277375
25   70:ee:50:15:e8:c8  19.621655
